# Amooora text data cleaning

## Import libraries and load csv

In [1]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
# nltk.download('punkt_tab')

In [2]:
!tree ..

..
├── notebooks
│   ├── data_cleaning.ipynb
│   ├── dedemenezes_exploratory.ipynb
│   ├── dedemenezes_plotting_topics.ipynb
│   ├── dedemenezes_text_classification_bert.ipynb
│   └── exploratory_analysis.ipynb
├── raw_data
│   ├── combined_text_preprocessed.csv
│   ├── okcupid_profiles.csv
│   ├── okcupid_profiles_encoded.csv
│   ├── parsed_data_public.csv
│   ├── question_data.csv
│   ├── text_and_topics.csv
│   └── user_data_public.csv
├── README.md
├── requirements.txt
└── test_req.txt

2 directories, 15 files


In [3]:
df = pd.read_csv('../raw_data/okcupid_profiles.csv')

In [4]:
df.shape

(59946, 31)

In [5]:
df.head(2)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN


## Access Text Dataset

In [6]:
df.iloc[:, :21]

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (..."
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no,"english, french, c++"
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,NaN,pisces,no,"english, german (poorly)"
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,likes dogs and likes cats,NaN,aquarius,no,english
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59,single,f,straight,NaN,NaN,socially,never,graduated from college/university,NaN,...,-1,sales / marketing / biz dev,2012-06-12-21-47,"oakland, california",has kids,has dogs,catholicism but not too serious about it,cancer and it&rsquo;s fun to think about,no,english
59942,24,single,m,straight,fit,mostly anything,often,sometimes,working on college/university,"white, other",...,-1,entertainment / media,2012-06-29-11-01,"san francisco, california",doesn't have kids,likes dogs and likes cats,agnosticism,leo but it doesn&rsquo;t matter,no,english (fluently)
59943,42,single,m,straight,average,mostly anything,not at all,never,graduated from masters program,asian,...,100000,construction / craftsmanship,2012-06-27-23-37,"south san francisco, california",doesn't have kids,NaN,christianity but not too serious about it,sagittarius but it doesn&rsquo;t matter,no,english (fluently)
59944,27,single,m,straight,athletic,mostly anything,socially,often,working on college/university,"asian, black",...,-1,medicine / health,2012-06-23-13-01,"san francisco, california","doesn't have kids, but wants them",likes dogs and likes cats,agnosticism but not too serious about it,leo and it&rsquo;s fun to think about,trying to quit,"english (fluently), spanish (poorly), chinese ..."


In [7]:
text_df = df.iloc[:, 21:]

In [8]:
text_df.head()

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


## Checking dataset

In [9]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   essay0  54458 non-null  object
 1   essay1  52374 non-null  object
 2   essay2  50308 non-null  object
 3   essay3  48470 non-null  object
 4   essay4  49409 non-null  object
 5   essay5  49096 non-null  object
 6   essay6  46175 non-null  object
 7   essay7  47495 non-null  object
 8   essay8  40721 non-null  object
 9   essay9  47343 non-null  object
dtypes: object(10)
memory usage: 4.6+ MB


### Duplicates

In [10]:
text_df.duplicated().sum()

2134

In [11]:
text_df[text_df.duplicated()]

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
59757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have `2134`duplicated observations all made of empty answers to all questions

In [12]:
# no_dup_df = text_df.drop_duplicates()
# no_dup_df.shape

### Empty values

In [13]:
text_df.isnull().sum().sort_values(ascending=False) / len(text_df) * 100

essay8    32.070530
essay6    22.972342
essay9    21.023922
essay7    20.770360
essay3    19.143896
essay5    18.099623
essay4    17.577486
essay2    16.077803
essay1    12.631368
essay0     9.154906
dtype: float64

In [14]:
text_df.dropna().shape

(29866, 10)

In [15]:
text_df.shape

(59946, 10)

Drop is not an option!

## % of answered questions

In [16]:
text_df.columns

Index(['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6',
       'essay7', 'essay8', 'essay9'],
      dtype='object')

In [17]:
text_df['not_answered_percent'] = text_df.apply(lambda x: x.isna().sum() / len(text_df.columns) * 100, axis=1)

In [18]:
text_df

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,not_answered_percent
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,0.0
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN,40.0
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",10.0
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,30.0
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...
59941,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,0.0
59942,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.,0.0
59943,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies: bourne serie...",- iphone - friends and family - internet - bay...,"aside from work, how to improve my home.",out enjoying friendly conversation over dinner.,please let me think about this more.,we have similar interests.,0.0
59944,"""all i have in this world are my balls and my ...","following my dreams... ""you got a dream... you...",listening,it used to be the hair until i mowed it off bu...,where to begin musically: right now i listen t...,"music, family, friends, a basketball, hoop, so...",what can i do to make someone chuckle....,what i would do on any other day. everydays a ...,i like walking around in other people's house ...,you are interested and interesting...,0.0


## Replace NaN with empty string

In [19]:
no_nan_df = text_df.replace(np.nan, '')

In [20]:
no_nan_df.isna()

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,not_answered_percent
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
59941,False,False,False,False,False,False,False,False,False,False,False
59942,False,False,False,False,False,False,False,False,False,False,False
59943,False,False,False,False,False,False,False,False,False,False,False
59944,False,False,False,False,False,False,False,False,False,False,False


## Combining into one big text column

In [21]:
no_nan_df['combined'] = no_nan_df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [22]:
no_nan_df.combined

0        about me:  i would love to think that i was so...
1        i am a chef: this is what that means. 1. i am ...
2        i'm not ashamed of much, but writing public te...
3        i work in a library and go to school. . . read...
4        hey how's it going? currently vague on the pro...
                               ...                        
59941    vibrant, expressive, caring optimist. i love b...
59942    i'm nick. i never know what to write about mys...
59943    hello! i enjoy traveling, watching movies, and...
59944    "all i have in this world are my balls and my ...
59945    is it odd that having a little "enemy" status ...
Name: combined, Length: 59946, dtype: object

## Cleaning text

In [23]:
def basic_cleaning(sentence):
    # Basic cleaning
    sentence = sentence.strip() # remove whitespaces
    sentence = sentence.lower() # lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) # remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '')
        
    return sentence

In [24]:
%%time
cleaned_df = pd.DataFrame()

# Criar Novas colunas essayX_clean
essays = no_nan_df.columns[:10]
for essay in essays:
    cleaned_df[f"{essay}_clean"] = no_nan_df[essay].apply(basic_cleaning)

cleaned_df['combined_clean'] = no_nan_df.combined.apply(basic_cleaning)

cleaned_df

CPU times: user 26.1 s, sys: 458 ms, total: 26.5 s
Wall time: 26.4 s


,essay0_clean,essay1_clean,essay2_clean,essay3_clean,essay4_clean,essay5_clean,essay6_clean,essay7_clean,essay8_clean,essay9_clean,combined_clean
0,about me i would love to think that i was som...,currently working as an international agent fo...,making people laugh ranting about a good salti...,the way i look i am a six foot half asian half...,books absurdistan the republic of mice and men...,food water cell phone shelter,duality and humorous things,trying to find someone to hang out with i am d...,i am new to california and looking for someone...,you want to be swept off your feet you are tir...,about me i would love to think that i was som...
1,i am a chef this is what that means i am a wo...,dedicating everyday to being an unbelievable b...,being silly having ridiculous amonts of fun wh...,,i am die hard christopher moore fan i dont rea...,delicious porkness in all of its glories my bi...,,,i am very open and will share just about anything,,i am a chef this is what that means i am a wo...
2,im not ashamed of much but writing public text...,i make nerdy software for musicians artists an...,improvising in different contexts alternating ...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,,viewing listening dancing talking drinking per...,when i was five years old i was known as the b...,you are bright open intense silly ironic criti...,im not ashamed of much but writing public text...
3,i work in a library and go to school,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,bataille celine beckett lynch jarmusch rw fa...,,cats and german philosophy,,,you feel so inclined,i work in a library and go to school reading...
4,hey hows it going currently vague on the profi...,work work work work play,creating imagery to look at httpbagsbrownblogs...,i smile a lot and my inquisitive nature,music bands rappers musicians at the moment th...,,,,,,hey hows it going currently vague on the profi...
...,...,...,...,...,...,...,...,...,...,...,...
59941,vibrant expressive caring optimist i love bein...,the happiest times have been when life came to...,i make an outstanding osso bucco i am also ver...,i am told that people notice my smile eyes and...,i am an avid movie watcher and follow the broa...,my family my dog italy words and music,writing my book,running with my dog finishing up the work week...,i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,vibrant expressive caring optimist i love bein...
59942,im nick i never know what to write about mysel...,currently finishing school for film production...,filmmaking photography graphic design web desi...,dude i dont know,movies hook the greatest adventure ever gladia...,iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus tofrom wor...,bringin home bacon or drinking and shakin,when i was i got a tattoo of waldo somewhere ...,meh if you made it this far you might as well,im nick i never know what to write about mysel...
59943,hello i enjoy traveling watching movies and ha...,im a civil engineer who enjoys helping the cit...,looking at things objectively getting things...,im quiet until i get used to the environment b...,last book game change movies bourne series act...,iphone friends and family internet bay are...,aside from work how to improve my home,out enjoying friendly conversation over dinner,please let me think about this more,we have similar interests,hello i enjoy traveling watching movies and ha...
59944,all i have in this world are my balls and my i...,following my dreams you got a dream you gotta ...,listening,it used to be the hair until i mowed it off bu...,where to begin musically right now i listen to...,music family friends a basketball hoop somethi...,what can i do to make someone chuckle,what i would do on any other day e

## Preprocessing

### Tokenization

Let's take one row and play with **Tokenization**, **stopwords** and **lemmatize** 

In [25]:
text = no_nan_df.loc[0, 'essay0']

word_tokens = word_tokenize(text)

pd.DataFrame({'essay0_words': word_tokens})

,essay0_words
0,about
1,me
2,:
3,i
4,would
...,...
264,a
265,good
266,attention
267,span


### Remove stopwords

In [26]:
word_tokens

stop_words = set(stopwords.words('english'))

pd.DataFrame()

stopwords_removed = [w for w in word_tokens if w in stop_words]

tokens_cleaned = [w for w in word_tokens if not w in stop_words]
tokens_cleaned

[':',
 'would',
 'love',
 'think',
 'kind',
 'intellectual',
 ':',
 'either',
 'dumbest',
 'smart',
 'guy',
 ',',
 'smartest',
 'dumb',
 'guy',
 '.',
 'ca',
 "n't",
 'say',
 'tell',
 'difference',
 '.',
 'love',
 'talk',
 'ideas',
 'concepts',
 '.',
 'forge',
 'odd',
 'metaphors',
 'instead',
 'reciting',
 'cliches',
 '.',
 'like',
 'simularities',
 'friend',
 'mine',
 "'s",
 'house',
 'underwater',
 'salt',
 'mine',
 '.',
 'favorite',
 'word',
 'salt',
 'way',
 '(',
 'weird',
 'choice',
 'know',
 ')',
 '.',
 'things',
 'life',
 'better',
 'metaphors',
 '.',
 'seek',
 'make',
 'little',
 'better',
 'everyday',
 ',',
 'productively',
 'lazy',
 'way',
 '.',
 'got',
 'tired',
 'tying',
 'shoes',
 '.',
 'considered',
 'hiring',
 'five',
 'year',
 'old',
 ',',
 'would',
 'probably',
 'tie',
 'shoes',
 '...',
 'decided',
 'wear',
 'leather',
 'shoes',
 'dress',
 'shoes',
 '.',
 ':',
 'love',
 'really',
 'serious',
 ',',
 'really',
 'deep',
 'conversations',
 'really',
 'silly',
 'stuff',
 '.

### Lemmatizing

In [27]:
# Lemmatizing the verbs
verb_lemmatized = [
    WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
    for word in tokens_cleaned
]

# 2 - Lemmatizing the nouns
noun_lemmatized = [
    WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
    for word in verb_lemmatized
]

pd.DataFrame(dict(token=tokens_cleaned, verbs=verb_lemmatized, noun=noun_lemmatized))

,token,verbs,noun
0,:,:,:
1,would,would,would
2,love,love,love
3,think,think,think
4,kind,kind,kind
...,...,...,...
150,lastly,lastly,lastly
151,good,good,good
152,attention,attention,attention
153,span,span,span


### All together

In [28]:
def basic_preprocessing(sentence):
    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n")
        for word in lemmatized
    ]
    
    cleaned_sentence = ' '.join(word for word in noun_lemmatized)
    return cleaned_sentence

text = cleaned_df.loc[0, 'essay0_clean']
print(f"Original text:\n{text}")
basic_preprocessing(text)

Original text:
about me  i would love to think that i was some some kind of intellectual either the dumbest smart guy or the smartest dumb guy cant say i can tell the difference i love to talk about ideas and concepts i forge odd metaphors instead of reciting cliches like the simularities between a friend of mines house and an underwater salt mine my favorite word is salt by the way weird choice i know to me most things in life are better as metaphors i seek to make myself a little better everyday in some productively lazy way got tired of tying my shoes considered hiring a five year old but would probably have to tie both of our shoes decided to only wear leather shoes dress shoes  about you  you love to have really serious really deep conversations about really silly stuff you have to be willing to snap me out of a light hearted rant with a kiss you dont have to be funny but you have to be able to make me laugh you should be able to bend spoons with your mind and telepathically make 

'would love think kind intellectual either dumbest smart guy smartest dumb guy cant say tell difference love talk idea concept forge odd metaphor instead recite cliche like simularities friend mine house underwater salt mine favorite word salt way weird choice know thing life better metaphor seek make little better everyday productively lazy way get tire tie shoe consider hire five year old would probably tie shoe decide wear leather shoe dress shoe love really serious really deep conversation really silly stuff will snap light hearted rant kiss dont funny able make laugh able bend spoon mind telepathically make smile still work love life cool let wind blow extra point read guess favorite video game hint give yet lastly good attention span'

In [29]:
# cleaned_df.essay0_clean.apply(basic_preprocessing)

In [30]:
# %%time
# clean_preprocessed_df = cleaned_df.apply(lambda row: row.apply(basic_preprocessing))

In [31]:
# %%time
# clean_preprocessed_df = cleaned_df.applymap(basic_preprocessing)

In [32]:
%%time
clean_preprocessed_df = cleaned_df['combined_clean'].apply(basic_preprocessing)

CPU times: user 3min 34s, sys: 2.87 s, total: 3min 37s
Wall time: 3min 38s


In [33]:
clean_preprocessed_df

0        would love think kind intellectual either dumb...
1        chef mean workaholic love cook regardless whet...
2        im ashamed much write public text online date ...
3        work library go school read thing write old de...
4        hey hows go currently vague profile know come ...
                               ...                        
59941    vibrant expressive care optimist love people t...
59942    im nick never know write im sure hand im south...
59943    hello enjoy travel watch movie hang friend rul...
59944    world ball integrity one take either away momm...
59945    odd little enemy status someone make seem inte...
Name: combined_clean, Length: 59946, dtype: object

In [34]:
new_names = dict(
#     essay0_clean="essay0_preprocessed",
#     essay1_clean="essay1_preprocessed",
#     essay2_clean="essay2_preprocessed",
#     essay3_clean="essay3_preprocessed",
#     essay4_clean="essay4_preprocessed",
#     essay5_clean="essay5_preprocessed",
#     essay6_clean="essay6_preprocessed",
#     essay7_clean="essay7_preprocessed",
#     essay8_clean="essay8_preprocessed",
#     essay9_clean="essay9_preprocessed",
    combined_clean="combined_preprocessed"
)
clean_preprocessed_df = clean_preprocessed_df.to_frame().rename(columns=new_names)

In [35]:
clean_preprocessed_df.combined_preprocessed.head(1)

0    would love think kind intellectual either dumb...
Name: combined_preprocessed, dtype: object

In [36]:
cleaned_df.combined_clean.head(1)

0    about me  i would love to think that i was som...
Name: combined_clean, dtype: object

In [37]:
no_nan_df.combined.head(1)

0    about me:  i would love to think that i was so...
Name: combined, dtype: object

In [38]:
no_nan_df

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,not_answered_percent,combined
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,0.0,about me: i would love to think that i was so...
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,,,i am very open and will share just about anyth...,,40.0,i am a chef: this is what that means. 1. i am ...
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",10.0,"i'm not ashamed of much, but writing public te..."
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",,cats and german philosophy,,,you feel so inclined.,30.0,i work in a library and go to school. . . read...
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",,,,,,50.0,hey how's it going? currently vague on the pro...
...,...,...,...,...,...,...,...,...,...,...,...,...
59941,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,0.0,"vibrant, expressive, caring optimist. i love b..."
59942,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.,0.0,i'm nick. i never know what to write about mys...
59943,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies: bourne serie...",- iphone - friends and family - internet - bay...,"aside from work, how to improve my home.",out enjoying friendly conversation over dinner.,please let me think about this more.,we have similar interests.,0.0,"hello! i enjoy traveling, watching movies, and..."
59944,"""all i have in this world are my balls and my ...","following my dreams... ""you got a dream... you...",listening,it used to be the hair until i mowed it off bu...,where to begin musically: right now i listen t...,"music, family, friends, a ba

In [39]:
clean_preprocessed_df['answered_percent'] = 100 - no_nan_df.not_answered_percent
clean_preprocessed_df.isna().sum()

combined_preprocessed    0
answered_percent         0
dtype: int64

## New text_length column

In [40]:
clean_preprocessed_df['text_length'] = clean_preprocessed_df.combined_preprocessed.str.len()
clean_preprocessed_df

,combined_preprocessed,answered_percent,text_length
0,would love think kind intellectual either dumb...,100.0,1565
1,chef mean workaholic love cook regardless whet...,60.0,815
2,im ashamed much write public text online date ...,90.0,3728
3,work library go school read thing write old de...,70.0,330
4,hey hows go currently vague profile know come ...,50.0,496
...,...,...,...
59941,vibrant expressive care optimist love people t...,100.0,1040
59942,im nick never know write im sure hand im south...,100.0,1634
59943,hello enjoy travel watch movie hang friend rul...,100.0,1189
59944,world ball integrity one take either away momm...,100.0,1122


## Saving into CSV

In [41]:
!tree ..

..
├── notebooks
│   ├── dedemenezes_exploratory.ipynb
│   └── demenezes_simple_RNN.ipynb
├── raw_data
│   ├── combined_text_preprocessed.csv
│   ├── okcupid_profiles.csv
│   ├── okcupid_profiles_encoded.csv
│   ├── parsed_data_public.csv
│   ├── question_data.csv
│   └── user_data_public.csv
├── README.md
└── requirements.txt

2 directories, 10 files


In [42]:
clean_preprocessed_df.to_csv('../raw_data/combined_text_preprocessed.csv', index=False, columns=clean_preprocessed_df.columns)

In [43]:
!tree ..

..
├── notebooks
│   ├── dedemenezes_exploratory.ipynb
│   └── demenezes_simple_RNN.ipynb
├── raw_data
│   ├── combined_text_preprocessed.csv
│   ├── okcupid_profiles.csv
│   ├── okcupid_profiles_encoded.csv
│   ├── parsed_data_public.csv
│   ├── question_data.csv
│   └── user_data_public.csv
├── README.md
└── requirements.txt

2 directories, 10 files


In [44]:
pd.read_csv('../raw_data/combined_text_preprocessed.csv',keep_default_na=False)

,combined_preprocessed,answered_percent,text_length
0,would love think kind intellectual either dumb...,100.0,1565
1,chef mean workaholic love cook regardless whet...,60.0,815
2,im ashamed much write public text online date ...,90.0,3728
3,work library go school read thing write old de...,70.0,330
4,hey hows go currently vague profile know come ...,50.0,496
...,...,...,...
59941,vibrant expressive care optimist love people t...,100.0,1040
59942,im nick never know write im sure hand im south...,100.0,1634
59943,hello enjoy travel watch movie hang friend rul...,100.0,1189
59944,world ball integrity one take either away momm...,100.0,1122


## Vectorizing

Using relative frequency rather than count is robust to document length

Takes into account the context of the whole corpus

In [41]:
clean_preprocessed_df.shape

(59946, 3)

In [42]:
clean_preprocessed_df.combined_preprocessed.values

array(['would love think kind intellectual either dumbest smart guy smartest dumb guy cant say tell difference love talk idea concept forge odd metaphor instead recite cliche like simularities friend mine house underwater salt mine favorite word salt way weird choice know thing life better metaphor seek make little better everyday productively lazy way get tire tie shoe consider hire five year old would probably tie shoe decide wear leather shoe dress shoe love really serious really deep conversation really silly stuff will snap light hearted rant kiss dont funny able make laugh able bend spoon mind telepathically make smile still work love life cool let wind blow extra point read guess favorite video game hint give yet lastly good attention span currently work international agent freight forward company import export domestic know work online class try better free time perhaps hour worth good book video game lazy sunday make people laugh rant good salt find simplicity complexity compl

In [43]:
%%time
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(min_df=0.1, ngram_range=(1,2))

# Training it on the texts
combined_weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(clean_preprocessed_df.combined_preprocessed.values).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

combined_weighted_words

CPU times: user 33.6 s, sys: 3.4 s, total: 37 s
Wall time: 37.7 s


,actually,adventure,almost,along,also,always,amaze,american,anything,area,...,woman,word,work,world,would,write,year,yes,yet,youre
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125752,0.000000,...,0.000000,0.103616,0.140483,0.000000,0.133960,0.000000,0.064378,0.000000,0.106134,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.117395,0.000000,0.000000,0.107094,0.000000,...,0.000000,0.000000,0.079760,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,0.000000,0.000000,0.000000,0.081198,0.000000,0.000000,0.000000,0.043892,0.000000,...,0.000000,0.000000,0.032689,0.049425,0.000000,0.057229,0.089881,0.000000,0.000000,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.187798,0.000000,0.000000,0.328780,0.000000,0.000000,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.212428,0.000000,0.000000,...,0.000000,0.000000,0.385680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,0.0,0.000000,0.000000,0.142332,0.078259,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.139426,0.126022,0.285816,0.000000,0.110315,0.000000,0.000000,0.000000,0.0
59942,0.0,0.084036,0.193807,0.000000,0.056493,0.066949,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.045486,0.000000,0.000000,0.159267,0.062534,0.102665,0.103094,0.0
59943,0.0,0.000000,0.103404,0.000000,0.060283,0.000000,0.000000,0.000000,0.065172,0.181377,...,0.000000,0.000000,0.048538,0.000000,0.069426,0.000000,0.133459,0.000000,0.110010,0.0
59944,0.0,0.000000,0.000000,0.000000,0.000000,0.079786,0.000000,0.000000,0.218355,0.000000,...,0.112628,0.000000,0.000000,0.081961,0.155072,0.000000,0.000000,0.000000,0.000000,0.0


Could run a cross validation but we do not have the y

### Vectorizing only bigrams

In [44]:
%%time
# Instantiating the TfidfVectorizer
bigram_tf_idf_vectorizer = TfidfVectorizer(min_df=0.1, ngram_range=(2,2))

# Training it on the texts
bigram_combined_weighted_words = pd.DataFrame(bigram_tf_idf_vectorizer.fit_transform(clean_preprocessed_df.combined_preprocessed.values).toarray(),
                 columns = bigram_tf_idf_vectorizer.get_feature_names_out())

bigram_combined_weighted_words

CPU times: user 17 s, sys: 1.54 s, total: 18.5 s
Wall time: 19.2 s


,bay area,family friend,friend family,get know,im pretty,make people,new thing,san francisco,sense humor,spend time,watch movie
0,0.000000,0.00000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.653205,0.000000,0.757181,0.000000
3,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
59941,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
59942,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000
59943,0.744222,0.00000,0.391524,0.0,0.0,0.0,0.0,0.358501,0.000000,0.000000,0.405362
59944,0.000000,1.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


## Latent Dirichlet Allocation

In [45]:
from sklearn.decomposition import LatentDirichletAllocation

### 💻 Finding the topics (2)

In [52]:
%%time

# Instantiate the LDA
n_components = 2
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fit the LDA on the vectorized documents
lda_model.fit(combined_weighted_words)

CPU times: user 12min 37s, sys: 86.8 ms, total: 12min 37s
Wall time: 12min 39s


LatentDirichletAllocation(max_iter=100, n_components=2)

In [53]:
combined_weighted_words

,actually,adventure,almost,along,also,always,amaze,american,anything,area,...,woman,word,work,world,would,write,year,yes,yet,youre
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125752,0.000000,...,0.000000,0.103616,0.140483,0.000000,0.133960,0.000000,0.064378,0.000000,0.106134,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.117395,0.000000,0.000000,0.107094,0.000000,...,0.000000,0.000000,0.079760,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,0.000000,0.000000,0.000000,0.081198,0.000000,0.000000,0.000000,0.043892,0.000000,...,0.000000,0.000000,0.032689,0.049425,0.000000,0.057229,0.089881,0.000000,0.000000,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.187798,0.000000,0.000000,0.328780,0.000000,0.000000,0.000000,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.212428,0.000000,0.000000,...,0.000000,0.000000,0.385680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,0.0,0.000000,0.000000,0.142332,0.078259,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.139426,0.126022,0.285816,0.000000,0.110315,0.000000,0.000000,0.000000,0.0
59942,0.0,0.084036,0.193807,0.000000,0.056493,0.066949,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.045486,0.000000,0.000000,0.159267,0.062534,0.102665,0.103094,0.0
59943,0.0,0.000000,0.103404,0.000000,0.060283,0.000000,0.000000,0.000000,0.065172,0.181377,...,0.000000,0.000000,0.048538,0.000000,0.069426,0.000000,0.133459,0.000000,0.110010,0.0
59944,0.0,0.000000,0.000000,0.000000,0.000000,0.079786,0.000000,0.000000,0.218355,0.000000,...,0.112628,0.000000,0.000000,0.081961,0.155072,0.000000,0.000000,0.000000,0.000000,0.0


#### Document Mixture (of Topics)

In [68]:
%%time
document_topic_mixture = lda_model.transform(combined_weighted_words)

document_topic_mixture

two_topics_df = pd.DataFrame({
    'topic_0_from_two': document_topic_mixture[:, 0],
    'topic_1_from_two': document_topic_mixture[:, 1],
    'original_text': clean_preprocessed_df.combined_preprocessed
})

CPU times: user 4.82 s, sys: 0 ns, total: 4.82 s
Wall time: 5.07 s


In [69]:
two_topics_df

,topic_0_from_two,topic_1_from_two,original_text
0,0.271588,0.728412,would love think kind intellectual either dumb...
1,0.446532,0.553468,chef mean workaholic love cook regardless whet...
2,0.280111,0.719889,im ashamed much write public text online date ...
3,0.330528,0.669472,work library go school read thing write old de...
4,0.450953,0.549047,hey hows go currently vague profile know come ...
...,...,...,...
59941,0.571123,0.428877,vibrant expressive care optimist love people t...
59942,0.218072,0.781928,im nick never know write im sure hand im south...
59943,0.705119,0.294881,hello enjoy travel watch movie hang friend rul...
59944,0.307921,0.692079,world ball integrity one take either away momm...


#### Topic Mixture (of Words)

In [55]:
topic_word_mixture = pd.DataFrame(
    lda_model.components_,
    columns = tf_idf_vectorizer.get_feature_names_out()
)

topic_word_mixture

,actually,adventure,almost,along,also,always,amaze,american,anything,area,...,woman,word,work,world,would,write,year,yes,yet,youre
0,1.007910,1195.564697,260.426651,196.783936,892.985046,801.870597,405.058278,331.945012,785.301092,1160.747223,...,294.037876,0.827736,2635.651287,753.815249,545.571153,3.678006,1308.205494,1.219521,14.389899,123.589083
1,844.700131,80.831254,536.235488,467.092895,1807.820898,1040.442785,347.313044,434.108523,1500.713380,1.059540,...,774.566054,769.961649,1225.458473,1060.768864,1496.359167,1512.436281,854.931790,701.281808,678.209551,1538.506653


##### Most relevant words for each topic

In [56]:
topic_word_mixture.iloc[0].sort_values(ascending = False).head(10)

love      4109.371506
friend    3899.225045
new       2760.198738
enjoy     2734.565391
work      2635.651287
music     2543.124120
family    2496.045852
food      2390.284481
good      2255.658250
movie     2180.590099
Name: 0, dtype: float64

In [57]:
topic_word_mixture.iloc[1].sort_values(ascending = False).head(10)

im        6404.652322
like      3924.483119
get       2979.343328
dont      2926.213422
thing     2769.339857
think     2709.495874
know      2585.952050
really    2581.374786
people    2311.412257
one       2301.403466
Name: 1, dtype: float64

In [58]:
lda_model.score(combined_weighted_words)

-2221826.73473652

### Finding the topics (5)

In [46]:
%%time

n_components = 5
five_lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)


five_lda_model.fit(combined_weighted_words)

CPU times: user 32min 9s, sys: 585 ms, total: 32min 9s
Wall time: 32min 10s


LatentDirichletAllocation(max_iter=100, n_components=5)

#### Document Mixture of Topics

In [47]:
%%time
five_document_topic_mixture = five_lda_model.transform(combined_weighted_words)


CPU times: user 16.1 s, sys: 17.7 ms, total: 16.1 s
Wall time: 15 s


In [55]:
five_document_topic_mixture

array([[0.0236401 , 0.02357332, 0.02390446, 0.02350866, 0.90537345],
       [0.0275766 , 0.16478151, 0.7524778 , 0.02746957, 0.02769452],
       [0.02130858, 0.02128501, 0.02120363, 0.02131398, 0.9148888 ],
       ...,
       [0.02415027, 0.02419021, 0.02426289, 0.90326256, 0.02413407],
       [0.02445752, 0.02447094, 0.02466304, 0.02438295, 0.90202556],
       [0.02207099, 0.02213029, 0.02221014, 0.02206768, 0.91152091]])

In [51]:
five_topics_df = pd.DataFrame({
    'topic_0_from_five': five_document_topic_mixture[:, 0],
    'topic_1_from_five': five_document_topic_mixture[:, 1],
    'topic_2_from_five': five_document_topic_mixture[:, 2],
    'topic_3_from_five': five_document_topic_mixture[:, 3],
    'topic_4_from_five': five_document_topic_mixture[:, 4],
})

#### Topic means

In [52]:
topic_means = five_topics_df.mean()
print(topic_means)

topic_0_from_five    0.179609
topic_1_from_five    0.182617
topic_2_from_five    0.170216
topic_3_from_five    0.167371
topic_4_from_five    0.300186
dtype: float64


#### Topic Mixture of Words

In [63]:
five_topic_word_mixture = pd.DataFrame(
    five_lda_model.components_,
    columns = tf_idf_vectorizer.get_feature_names_out()
)

five_topic_word_mixture

,actually,adventure,almost,along,also,always,amaze,american,anything,area,...,woman,word,work,world,would,write,year,yes,yet,youre
0,486.256400,260.873365,303.607542,258.075512,1037.312061,639.996142,243.433413,207.766340,794.414262,245.860250,...,339.150579,423.392056,1078.332096,578.826448,807.979118,694.974833,689.988979,355.065164,313.074742,792.456199
1,85.841302,346.167153,109.107549,118.817171,481.024685,327.670131,169.491502,95.673243,303.049932,163.882557,...,421.242422,113.699207,635.476394,480.478907,446.087779,252.231860,335.088149,103.294965,166.055327,197.675425
2,125.452062,280.969248,139.313031,89.989400,442.197515,292.437122,125.318313,292.592724,478.916099,96.105311,...,111.386765,149.003246,612.556464,340.062640,238.445882,329.053674,317.396181,122.093172,85.277942,305.979426
3,54.726448,122.134791,109.009119,102.312471,324.522957,272.805489,49.836048,71.483609,387.337693,69.536246,...,136.322516,32.698429,774.534516,134.079374,300.447708,102.856498,206.522877,44.129776,50.940442,162.690612
4,93.431830,266.251394,135.624899,94.682276,415.748726,309.404499,164.292045,98.537619,322.296486,586.422399,...,60.501647,51.996446,760.210290,281.136744,248.969833,136.997421,614.141098,77.918252,77.250995,203.294073


### Saving topics into CSV

In [66]:
clean_preprocessed_df.head()

,combined_preprocessed,answered_percent,text_length
0,would love think kind intellectual either dumb...,100.0,1565
1,chef mean workaholic love cook regardless whet...,60.0,815
2,im ashamed much write public text online date ...,90.0,3728
3,work library go school read thing write old de...,70.0,330
4,hey hows go currently vague profile know come ...,50.0,496


In [76]:
combined_topics_df = pd.concat([clean_preprocessed_df, five_topics_df, two_topics_df], axis=1).drop(columns=['original_text'])

In [78]:
combined_topics_df.to_csv('../raw_data/text_and_topics.csv', index=False, columns=combined_topics_df.columns)

In [79]:
test_df = pd.read_csv('../raw_data/text_and_topics.csv', keep_default_na=False)
test_df

,combined_preprocessed,answered_percent,text_length,topic_0_from_five,topic_1_from_five,topic_2_from_five,topic_3_from_five,topic_4_from_five,topic_0_from_two,topic_1_from_two
0,would love think kind intellectual either dumb...,100.0,1565,0.905500,0.023574,0.023639,0.023778,0.023509,0.271588,0.728412
1,chef mean workaholic love cook regardless whet...,60.0,815,0.027683,0.339134,0.027590,0.578125,0.027468,0.446532,0.553468
2,im ashamed much write public text online date ...,90.0,3728,0.914891,0.021284,0.021307,0.021207,0.021311,0.280111,0.719889
3,work library go school read thing write old de...,70.0,330,0.045781,0.045620,0.817402,0.045673,0.045524,0.330528,0.669472
4,hey hows go currently vague profile know come ...,50.0,496,0.031946,0.031976,0.872147,0.032119,0.031812,0.450953,0.549047
...,...,...,...,...,...,...,...,...,...,...
59941,vibrant expressive care optimist love people t...,100.0,1040,0.028120,0.887424,0.028228,0.028106,0.028123,0.571123,0.428877
59942,im nick never know write im sure hand im south...,100.0,1634,0.900146,0.024829,0.025074,0.024849,0.025101,0.218072,0.781928
59943,hello enjoy travel watch movie hang friend rul...,100.0,1189,0.024132,0.024193,0.024150,0.024269,0.903256,0.705119,0.294881
59944,world ball integrity one take either away momm...,100.0,1122,0.902030,0.024473,0.024496,0.024619,0.024383,0.307921,0.692079


### Finding the topics (10) 

In [54]:
%%time

n_components = 10
ten_lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)


ten_lda_model.fit(combined_weighted_words)

CPU times: user 14min 55s, sys: 150 ms, total: 14min 55s
Wall time: 14min 55s


LatentDirichletAllocation(max_iter=100)

#### Document Mixture of Topics

In [55]:
%%time
ten_document_topic_mixture = ten_lda_model.transform(combined_weighted_words)

ten_topics = {}
for index in range(ten_document_topic_mixture.shape[1]):
    ten_topics[f"topic_{index}"] = ten_document_topic_mixture[:, index]
    
ten_topics

CPU times: user 8.23 s, sys: 0 ns, total: 8.23 s
Wall time: 7.24 s


{'topic_0': array([0.01144798, 0.01335621, 0.01031735, ..., 0.0117379 , 0.01183464,
        0.01073903]),
 'topic_1': array([0.01144787, 0.01335477, 0.01031793, ..., 0.01173906, 0.01183428,
        0.01073802]),
 'topic_2': array([0.89696629, 0.01335545, 0.01031684, ..., 0.01173731, 0.01183511,
        0.01073848]),
 'topic_3': array([0.011448  , 0.01335556, 0.01031708, ..., 0.01173817, 0.01183512,
        0.01073853]),
 'topic_4': array([0.0114489 , 0.01335492, 0.90714527, ..., 0.01173733, 0.01183695,
        0.90335547]),
 'topic_5': array([0.01144814, 0.01335488, 0.0103169 , ..., 0.01173728, 0.01183433,
        0.01073812]),
 'topic_6': array([0.01144837, 0.87980156, 0.01031722, ..., 0.01173746, 0.01183464,
        0.01073837]),
 'topic_7': array([0.01144809, 0.01335702, 0.01031717, ..., 0.01173743, 0.01183486,
        0.010738  ]),
 'topic_8': array([0.01144784, 0.01335487, 0.01031692, ..., 0.8943606 , 0.01183477,
        0.01073798]),
 'topic_9': array([0.01144852, 0.01335475, 0.0

In [57]:
ten_topics_df = pd.DataFrameme(ten_topics)
ten_topic_means = ten_topics_df.mean()
print(ten_topic_means)

topic_0    0.063823
topic_1    0.084326
topic_2    0.104312
topic_3    0.105723
topic_4    0.152280
topic_5    0.081898
topic_6    0.100417
topic_7    0.114937
topic_8    0.077470
topic_9    0.114814
dtype: float64


#### Topic mixture of words

In [58]:
ten_topic_word_mixture = pd.DataFrame(
    ten_lda_model.components_,
    columns = tf_idf_vectorizer.get_feature_names_out()
)

### Print topics

In [53]:
def print_topics(lda_model, vectorizer, top_words):
    # 1. TOPIC MIXTURE OF WORDS FOR EACH TOPIC
    topic_mixture = pd.DataFrame(
        lda_model.components_,
        columns = vectorizer.get_feature_names_out()
    )

    # 2. FINDING THE TOP WORDS FOR EACH TOPIC
    ## Number of topics
    n_components = topic_mixture.shape[0]

    ## Top words for each topic
    for topic in range(n_components):
        print("-"*10)
        print(f"For topic {topic}, here are the the top {top_words} words with weights:")

        topic_df = topic_mixture.iloc[topic]\
            .sort_values(ascending = False).head(top_words)

        print(round(topic_df,3))

#### Top words for five topics

In [54]:
print_topics(five_lda_model, tf_idf_vectorizer, 10)

----------
For topic 0, here are the the top 10 words with weights:
im       1252.590
music     990.187
like      973.469
love      833.585
movie     767.031
food      744.089
book      716.033
make      656.943
good      627.301
go        624.253
Name: 0, dtype: float64
----------
For topic 1, here are the the top 10 words with weights:
love      1396.362
life      1063.629
im        1045.349
like       886.775
friend     846.007
good       845.675
enjoy      842.009
people     708.719
time       689.118
music      680.050
Name: 1, dtype: float64
----------
For topic 2, here are the the top 10 words with weights:
im        1347.905
friend    1070.608
love      1035.885
like       894.365
family     804.148
work       787.951
go         779.071
good       715.900
movie      676.219
music      628.763
Name: 2, dtype: float64
----------
For topic 3, here are the the top 10 words with weights:
im               1173.667
love              924.824
san               834.533
new               

### Finding topics for Bigrams only (10 topics)

In [ ]:
%%time

n_components = 10
bigram_lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)


bigram_lda_model.fit(bigram_combined_weighted_words)

#### Most relevant words for each topic

In [ ]:
print_topics(bigram_lda_model, bigram_tf_idf_vectorizer, 5)